# 기계학습 (1221)

김보람  
2022-12-21

# imports

In [ ]:
import torch
import numpy as np 
import pandas as pd
from fastai.collab import * 

# 나는 솔로

## 주절주절 intro

`-` Data

In [ ]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv',index_col=0)
df_view 

`-` 데이터를 이해할때 필요한 가정들 – 내맘대로 한 설정임.

-   (옥순,영자,정숙)은 (영식,영철,영호)와 성격이 잘 맞고
    (영숙,순자,현숙)은 (광수,상철,영수)와 성격이 잘맞음
-   ((옥순,영자,정숙),(영식,영철,영호))은 MBTI가 I로 시작하고
    ((영숙,순자,현숙),(광수,상철,영수))는 MBTI가 E로 시작한다.

`-` 목표: NaN 을 추론

`-` 수동추론:

-   (옥순,영호)이 만난다면? $\to$ 둘다 I성향이니까 잘 맞지 않을까? $\to$
    4.0 정도?
-   (정숙,영식)조합은? $\to$ 둘다 I성향이니까 잘 맞지 않을까? + 정숙은
    다 잘맞던데..? $\to$ 4.8 정도?
-   (현숙,영식)조합은? $\to$ 현숙은 E성향인데 영식은 I성향이므로 잘
    안맞을 것임 + 현숙은 원래 좀 눈이 높음 $\to$ 0.25 정도?

`-` 좀 더 체계적인 추론

사람들이 가지고 있는 성향들을 두 개의 숫자로 표현하자.

-   옥순의 성향 = (I성향,E성향) = (1.9, 0.0)
-   영식의 성향 = (I성향,E성향) = (2.0, 0.1)
-   현숙의 성향 = (I성향,E성향) = (0.0, 1.5)

**(1)** 옥순과 영식의 궁합 $\approx$ 옥순의I성향$\times$영식의I성향 $+$
옥순의E성향$\times$영식의E성향 // 적합

In [ ]:
a1= np.array([1.9,0.0]).reshape(2,1) # a1은 옥순의 성향, col-vec으로 선언하자. 
b1= np.array([2.0,0.1]).reshape(2,1) # b1은 영식의 성향, col-vec으로 선언하자.
(a1*b1).sum()

**(2)** 현숙과 영식의 궁합 $\approx$ 현숙의I성향$\times$영식의I성향 $+$
현숙의E성향$\times$영식의E성향 // 예측

In [ ]:
a6= np.array([0.0,1.5]).reshape(2,1)
(a6*b1).sum()

-   그럴듯함..

`-` 모델링

아래가 같음을 관찰하라. (차원만 다름)

In [ ]:
(a1*b1).sum(), a1.T@b1

In [ ]:
(a6*b1).sum(), a6.T@b1

만약에 여자의성향, 남자의성향을 적당한 매트릭스로 정리할 수 있다면
궁합매트릭스를 만들 수 있음

In [ ]:
a1= np.array([1.9,0.0]).reshape(2,1)
a2= np.array([2.0,0.1]).reshape(2,1)
a3= np.array([2.5,1.0]).reshape(2,1)
a4= np.array([0.1,1.9]).reshape(2,1)
a5= np.array([0.2,2.1]).reshape(2,1)
a6= np.array([0.0,1.5]).reshape(2,1)
A = np.concatenate([a1,a2,a3,a4,a5,a6],axis=1)
A

In [ ]:
b1= np.array([2.0,0.1]).reshape(2,1)
b2= np.array([1.9,0.2]).reshape(2,1)
b3= np.array([1.8,0.3]).reshape(2,1)
b4= np.array([0.3,2.1]).reshape(2,1)
b5= np.array([0.2,2.0]).reshape(2,1)
b6= np.array([0.1,1.9]).reshape(2,1)
B = np.concatenate([b1,b2,b3,b4,b5,b6],axis=1)
B

In [ ]:
A.T@B

In [ ]:
a1.T@b1, a2.T@b2, a3.T@b1

결국 모형은 아래와 같다.

$$\text{궁합매트릭스} = {\bf A}^\top {\bf B} + \text{오차}$$

`-` 학습전략: 아래의 매트릭스중에서 어떤값은 관측하였고 어떤값은
관측하지 못함 $\to$ 관측한 값들만 대충 비슷하게 하면 되는거 아니야?

In [ ]:
A.T@B 

In [ ]:
df_view

`-` 자료를 아래와 같이 정리한다면?

In [ ]:
df = pd.DataFrame([(f,m,df_view.loc[f,m]) for f in df_view.index for m in df_view.columns if not np.isnan(df_view.loc[f,m])])
df.columns = ['X1','X2','y']
df

In [ ]:
mapp1 = {k[1]:k[0] for k in enumerate(df.X1.unique())}
mapp2 = {k[1]:k[0] for k in enumerate(df.X2.unique())}
mapp1,mapp2

In [ ]:
X1 = torch.tensor(list(map(lambda name: mapp1[name], df.X1)))
X2 = torch.tensor(list(map(lambda name: mapp2[name], df.X2)))
X1 = torch.nn.functional.one_hot(X1).float()
X2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df.y).float()

In [ ]:
X1

`-` yhat을 구하는 과정..

In [ ]:
l1 = torch.nn.Linear(in_features=6,out_features=2) # I성향 E성향.. #여출 
l2 = torch.nn.Linear(in_features=6,out_features=2) # 남출

In [ ]:
l1(X1) # 옥순~현숙의 성향들 

In [ ]:
l2(X2) # 영식~영수의 성향들 

`-` 몇개의 관측치만 생각해보자..

In [ ]:
df.head()

In [ ]:
(l1(X1)[0]*l2(X2)[0]).sum() # (옥순의성향 * 영식의성향).sum()

-   이 값이 실제로는 3.9 이어야 한다.

In [ ]:
(l1(X1)[1]*l2(X2)[1]).sum() # (옥순의성향 * 영철의성향).sum()

-   이 값이 실제로는 4.1 이어야 한다.

`-` yhat을 구하면!

In [ ]:
yhat = (l1(X1) * l2(X2)).sum(axis=1) # (l1(X1) * l2(X2)).sum(1)와 결과가 같음 
yhat

In [ ]:
yhat[:2],y[:2] # 이 값들이 비슷해야 하는데..

`-` 0~5 까지의 범위로 고정되어 있으니까 아래와 같이 해도 되겠음..

In [ ]:
sig = torch.nn.Sigmoid() # range: 0~1

In [ ]:
yhat = sig((l1(X1) * l2(X2)).sum(axis=1))*5 # (l1(X1) * l2(X2)).sum(1)와 결과가 같음    #range: 0~5
yhat

In [ ]:
loss = torch.mean((y-yhat)**2)
loss

## torch를 이용한 학습

In [ ]:
torch.manual_seed(43052)
l1 = torch.nn.Linear(6,2) 
l2 = torch.nn.Linear(6,2)
sig = torch.nn.Sigmoid() 

In [ ]:
loss_fn = torch.nn.MSELoss() 
optimizr = torch.optim.Adam(list(l1.parameters())+list(l2.parameters()))

In [ ]:
for epoc in range(5000):
    ## 1 
    feature1 = l1(X1)
    feature2 = l2(X2) 
    matching_score = (feature1*feature2).sum(axis=1) 
    yhat = sig(matching_score)*5 # 만약에 1~3점이라면 "1+sig(matching_score)*2" 와 같이 하면 되었을듯 
    ## 2 
    loss = loss_fn(yhat,y)    
    ## 3 
    loss.backward()    
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

In [ ]:
yhat

In [ ]:
y

In [ ]:
l1(X1) # 두번째 칼럼이 I 성향 점수로 "해석"된다

-   포인트: 여성출연자중, 정숙은 대체로 잘 맞춰주고 현숙은 그렇지
    않았음.. $\to$ 그러한 가중치가 잘 드러남!!

## fastai를 이용한 학습

`(1)` dls

In [ ]:
df.head() # 앞단계 전처리의 산물

In [ ]:
dls = CollabDataLoaders.from_df(df,bs=2,valid_pct=2/24) #bs:배치사이즈

`(2)` lrnr 생성

In [ ]:
lrnr = collab_learner(dls,n_factors=2,y_range=(0,5))

`(3)` 학습

In [ ]:
lrnr.fit(30,lr=0.05)

`(4)` 예측

**적합값 확인**

In [ ]:
lrnr.show_results()

**(옥순의 궁합)**

In [ ]:
df_new = pd.DataFrame({'X1':['옥순']*6, 'X2':['영식','영철','영호','광수','상철','영수']})
df_new

In [ ]:
lrnr.get_preds(dl=dls.test_dl(df_new))

비교를 위해서

In [ ]:
df_view

**(정숙의 궁합)**

In [ ]:
df_new = pd.DataFrame({'X1':['정숙']*6, 'X2':['영식','영철','영호','광수','상철','영수']})
df_new

In [ ]:
lrnr.get_preds(dl=dls.test_dl(df_new))

비교를 위해서

In [ ]:
df_view

`-` Appedix: fastai 구조공부..

In [ ]:
lrnr.model

In [ ]:
lrnr.model.forward??

-   bias를 제외하면 우리가 짠 모형과 같음!

# 커피 or 홍차

## data

`-` 예전에 살펴본 예제

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/main/posts/I.%20Overview/2022-09-08-rcmd_anal.csv')
df

`-` 기억을 살리기 위해서..

In [ ]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/main/posts/I.%20Overview/2022-09-08-rcmd_view.csv')
df_view

## 모형

(편의상 바이어스를 제외하면)

`-` 특징벡터:

-   유저1의 취향 = \[커피를 좋아하는 정도, 홍차를 좋아하는 정도\]
-   아이템1의 특징 = \[커피의 특징, 홍차인 특징\]

`-` 평점

-   유저1이 아이템1을 먹었을경우 평점: 유저1의 취향과 아이템1의 특징의
    내적 = (유저1의 취향 $\odot$ 아이템1의 특징).sum()

## 학습

`(1)` dls

In [ ]:
dls = CollabDataLoaders.from_df(df)

In [ ]:
dls.items

`(2)` lrnr

In [ ]:
lrnr = collab_learner(dls,n_factors=2) # 교재에는 y_range 를 설정하도록 되어있지만 설정 안해도 적합에는 크게 상관없음..

`(3)` fit

In [ ]:
lrnr.fit(10,0.1)

`(4)` predict

(적합된 값 확인)

In [ ]:
lrnr.show_results() # 누를때마다 결과다름

(예측값)

In [ ]:
df_new = pd.DataFrame({'user':[1,1,1,1], 'item':[9,10,11,12]})
df_new

In [ ]:
lrnr.get_preds(dl=dls.test_dl(df_new))